In [ ]:
train_data = pd.read_csv("./data/train.csv")

In [ ]:
### Age, Year-Of-Publication

from sklearn.preprocessing import MinMaxScaler

features_to_scale = ['Age', 'Year-Of-Publication']

scaler = MinMaxScaler()

train_data[features_to_scale] = scaler.fit_transform(train_data[features_to_scale])

In [ ]:
### Location  // nunique() > 442 > mapping : 151

train_data['Location'] = train_data['Location'].apply(lambda x: x.split(', ')[-1].strip())

with open("country_name_mapping.pkl", "rb") as f:
    country_name_mapping = pickle.load(f)
    
train_data['Location'] = train_data['Location'].apply(lambda x: country_name_mapping.get(x, x))
    
le = LabelEncoder()
train_data['Location_encoded'] = le.fit_transform(train_data['Location'])

embedding_layer = nn.Embedding(num_embeddings=151, embedding_dim=25)

location_embeddings = embedding_layer(torch.arange(151))


In [ ]:
location_embeddings_matrix = np.zeros((num_nodes, 25))

for i, location_encoded in enumerate(train_data['Location_encoded']):
    location_embeddings_matrix[i] = location_embeddings[location_encoded].numpy()

X_new = np.concatenate([X, location_embeddings_matrix], axis=1)

In [ ]:
### User-ID & Book-ID

# User-ID 열의 unique한 값들을 리스트로 만들기  > 83256
unique_user_ids = train_data['User-ID'].unique().tolist()

# Book-ID 열의 unique한 값들을 리스트로 만들기 > 243441> Q. 그럼 unique한 book name도 숫자가 같아야지?
unique_book_ids = train_data['Book-ID'].unique().tolist()

In [ ]:
# unique_user_ids 리스트를 기반으로 DataFrame 생성
unique_user_ids_df = pd.DataFrame(unique_user_ids, columns=['User-ID'])
# 내림차순 정렬
sorted_unique_user_ids_df = unique_user_ids_df.sort_values(by='User-ID', ascending=True)
# # 인덱스를 새 column으로 추가
sorted_unique_user_ids_df.reset_index(inplace=True, drop=False)
sorted_unique_user_ids_df.rename(columns={'index': 'UserNodeID'}, inplace=True)

# unique_book_ids 리스트를 기반으로 DataFrame 생성
unique_book_ids_df = pd.DataFrame(unique_book_ids, columns=['Book-ID'])
# 내림차순 정렬
sorted_unique_book_ids_df = unique_book_ids_df.sort_values(by='Book-ID', ascending=True)
# #인덱스를 새 column으로 추가
sorted_unique_book_ids_df.reset_index(inplace=True, drop=True)
sorted_unique_book_ids_df.reset_index(inplace=True, drop=False)
sorted_unique_book_ids_df.rename(columns={'index': 'BookNodeID'}, inplace=True)

# 인덱스 시작 번호를 83256으로 설정
sorted_unique_book_ids_df['BookNodeID'] += 83256

# UserNodeID  0 ~ 83255
# BookNodeID   83256 ~ 326696

In [ ]:
def make_ID_dict(df):
    ID_dict = {}
    for index, row in df.iterrows():
        ID_dict[row[1]] = row[0]
    return ID_dict

UserNodeID_dict = make_ID_dict(sorted_unique_user_ids_df)
BookNodeID_dict = make_ID_dict(sorted_unique_book_ids_df)

train_map = train_data.copy()

####### 매핑 진행
train_map['User-ID'] = train_map['User-ID'].map(UserNodeID_dict)
train_map['Book-ID'] = train_map['Book-ID'].map(BookNodeID_dict)

# edge_index

In [ ]:
edge_index = torch.tensor(train_map[['User-ID', 'Book-ID']].values, dtype=torch.long).t().contiguous()

# node feature

num_user_nodes = len(unique_user_ids)
num_book_nodes = len(unique_book_ids)